In [8]:
import pandas as pd
import pyodbc 
from datetime import date,datetime, timedelta
from pandas.tseries.offsets import MonthEnd
print('Librerias Cargadas.')
server = 'ECBPPRQ121,11124' 
database = 'DATALAB_IDN' 
username = 'mencarna' 
password = 'Pelus@141516' 
stringConn ='DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password+';Trusted_Connection=True;';
cnxn = pyodbc.connect(stringConn)
cursor = cnxn.cursor()
print('Conección Exitosa.')
Anio=2025
Mes=11
print('Variables del proceso: Anio: '+str(Anio)+', Mes: '+ str(Mes)+'.')

Librerias Cargadas.
Conección Exitosa.
Variables del proceso: Anio: 2025, Mes: 11.


In [9]:
query = f"""
select * from [DATALAB_IDN].[HIS].[Banca_Pes]
where year(FECHA_CORTE)={Anio} and month(FECHA_CORTE)={Mes}
"""
df_BSC = pd.read_sql(query, cnxn)
print('La obtención del BSC Historico es Exitosa.')

La obtención del BSC Historico es Exitosa.


C:\Users\mencarna\AppData\Local\Temp\ipykernel_26364\3651788336.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_BSC = pd.read_sql(query, cnxn)


In [10]:
df = pd.read_excel('//ECUIO197H02/group_files/Inteligencia de Negocios/IDN-Global/Pymes/PYMES Ajustes.xlsx',
                   header=[0,1,2],
                   index_col=[0],
                   sheet_name='Ajustes')
df = df.stack(level=1).stack(level=1).stack().unstack(level=1)
df = df.reset_index()
df = df[df.columns.difference(['level_2'])]
df=df.rename(columns = {'USUARIO':'PRODUCTO','level_0':'USUARIO'})
print(df.dtypes)

AJUSTES     float64
PRODUCTO     object
USUARIO      object
dtype: object


C:\Users\mencarna\AppData\Local\Temp\ipykernel_26364\956862453.py:5: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level=1).stack(level=1).stack().unstack(level=1)
C:\Users\mencarna\AppData\Local\Temp\ipykernel_26364\956862453.py:5: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level=1).stack(level=1).stack().unstack(level=1)


In [11]:
df = pd.merge(df_BSC, df,  how='left', 
             left_on=['USUARIO','PRODUCTO'], 
             right_on =['USUARIO','PRODUCTO'] )
df["AJUSTES"].fillna(0, inplace=True)
df["Real"]= df["Real"]+df["AJUSTES"]
df = df[df.columns.difference(['AJUSTES'])]
#df[df['Real','PRODUCTO','AJUSTES']]

C:\Users\mencarna\AppData\Local\Temp\ipykernel_26364\472455791.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["AJUSTES"].fillna(0, inplace=True)


In [12]:
len(df["PRODUCTO"].unique())

8

In [13]:
cursor.execute("TRUNCATE TABLE [DATALAB_IDN].[IDN].[Banca_Pes]")
for index, row in df.iterrows():
    cursor.execute("""INSERT INTO [DATALAB_IDN].[IDN].[Banca_Pes] 
    (PRODUCTO,USUARIO,Real,METAS,CATEGORIA,ACTIVO,TECHO_PRODUCTO,TECHO_P,TECHO_C,PORCENTAJE,INVERTIBILIDAD,
    TIPO,VALOR,ERROR,FECHA_CORTE,REGION,CodigoAgEjecucion,CodigoAgencia,Zona,AgenciaEjecucion,Agencia,
    Asesor,Medibles,PorcMedicion,ProvinciaAgencia,CiudadAgencia,CategorizacionAgencia,TipoMedicion,
    TipoAsesorProductividad) 
    values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""",
                   row.PRODUCTO,row.USUARIO,row.Real,row.METAS,row.CATEGORIA,row.ACTIVO,row.TECHO_PRODUCTO,
                   row.TECHO_P,row.TECHO_C,row.PORCENTAJE,row.INVERTIBILIDAD,row.TIPO,row.VALOR,row.ERROR,
                   row.FECHA_CORTE,row.REGION,row.CodigoAgEjecucion,row.CodigoAgencia,row.Zona,
                   row.AgenciaEjecucion,row.Agencia,row.Asesor,row.Medibles,row.PorcMedicion,
                   row.ProvinciaAgencia,row.CiudadAgencia,row.CategorizacionAgencia,row.TipoMedicion,
                   row.TipoAsesorProductividad)
    cnxn.commit()
print('Completado')

Completado


In [14]:
import pandas as pd
import pyodbc 
from datetime import date,datetime, timedelta
from pandas.tseries.offsets import MonthEnd

server = 'ECBPPRQ121,11124' 
database = 'DATALAB_IDN' 
username = 'erisimba' 
password = 'Picachuevil300#' 
stringConn ='DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password+';Trusted_Connection=True;';
cnxn = pyodbc.connect(stringConn)
cursor = cnxn.cursor()
cursor.execute(f"""
	DELETE FROM [DATALAB_IDN].[HIS].[Banca_Pes]
	WHERE YEAR(FECHA_CORTE) = {Anio} AND MONTH(FECHA_CORTE) = {Mes}
	INSERT INTO [DATALAB_IDN].[HIS].[Banca_Pes] 
	SELECT * FROM [DATALAB_IDN].[IDN].[Banca_Pes]
""")
cnxn.commit()
print('Ya esta tu webada. ')

Ya esta tu webada. 
